<a href="https://colab.research.google.com/github/wjcgithub/ai/blob/main/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gensim
from gensim.models import word2vec
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2019-08-18 15:52:00--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.97.141
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.97.141|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  16.5MB/s    in 98s     

2019-08-18 15:53:39 (16.1 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [ ]:
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz'
word_vectors = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
v_apple = word_vectors["apple"]
v_mango = word_vectors["mango"]
print(v_apple.shape)
print(v_mango.shape)
cosine_similarity([v_mango],[v_apple])

In [ ]:
import numpy as np
def odd_one_out(words):

    all_word_vectors = [word_vectors[w] for w in words]
    avg_vector = np.mean(all_word_vectors,axis=0)
    #print(avg_vector.shape)

    odd_one_out = None
    min_similarity = 1.0

    for w in words:
        sim = cosine_similarity([word_vectors[w]],[avg_vector])
        if sim < min_similarity:
            min_similarity = sim
            odd_one_out = w

        print("Similairy btw %s and avg vector is %.2f"%(w,sim))

    return odd_one_out



In [ ]:
pred=[]
import pandas as pd
import numpy as np
dfx = pd.read_csv('https://raw.githubusercontent.com/shankygupta79/GoogleCollabFiles/master/OddOne.csv').values
for i in range(4,8):
    a=dfx[i]
    a=a.reshape(-1)
    pred.append(odd_one_out(a))


print(pred)

(300,)
Similairy btw India and avg vector is 0.41
Similairy btw football and avg vector is 0.70
Similairy btw hockey and avg vector is 0.76
Similairy btw cricket and avg vector is 0.80
Similairy btw swimming and avg vector is 0.57
(300,)
Similairy btw who and avg vector is 0.54
Similairy btw why and avg vector is 0.81
Similairy btw what and avg vector is 0.79
Similairy btw where and avg vector is 0.67
Similairy btw is and avg vector is 0.47
(300,)
Similairy btw on and avg vector is 0.66
Similairy btw in and avg vector is 0.72
Similairy btw over and avg vector is 0.64
Similairy btw their and avg vector is 0.60
Similairy btw was and avg vector is 0.54
(300,)
Similairy btw India and avg vector is 0.79
Similairy btw Australia and avg vector is 0.71
Similairy btw Japan and avg vector is 0.79
Similairy btw Russia and avg vector is 0.74
Similairy btw China and avg vector is 0.82
['India', 'is', 'was', 'Australia']


## 2. Word Analogies Task
man -> woman ::     prince -> princess

italy -> italian ::     spain -> spanish

india -> delhi ::     japan -> tokyo

man -> woman ::     boy -> girl

small -> smaller ::     large -> larger

In [ ]:
def predict_word(a,b,c,word_vectors):
    """Accepts a triad of words, a,b,c and returns d such that a is to b : c is to d"""
    a,b,c = a.lower(),b.lower(),c.lower()

    # similarity |b-a| = |d-c| should be max
    max_similarity = -100

    d = None

    words = word_vectors.vocab.keys()

    wa,wb,wc = word_vectors[a],word_vectors[b],word_vectors[c]

    #to find d s.t similarity(|b-a|,|d-c|) should be max

    for w in words:
        if w in [a,b,c]:
            continue

        wv = word_vectors[w]
        sim = cosine_similarity([wb-wa],[wv-wc])

        if sim > max_similarity:
            max_similarity = sim
            d = w

    return d

In [ ]:
triad_2 = ("man","woman","prince")
predict_word(*triad_2,word_vectors)

'princess'

In [ ]:
#Inbuilt
word_vectors.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('queen', 0.7118192911148071)]